In [2]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from collections import Counter
import json

# Model Regresji Logistycznej (Logistic Regression)

## Zestaw danych

In [3]:
try:
    dataset = fetch_ucirepo(id=519)
    features = dataset.data.features[["age", "serum_creatinine", "ejection_fraction"]]
    target = dataset.data.targets
except Exception as e:
    df = pd.read_csv(
        "heart_failure_clinical_records_dataset.csv"
    ) 
    df.columns = [col.lower() if col == "DEATH_EVENT" else col for col in df.columns]
    features = df[["age", "serum_creatinine", "ejection_fraction"]]
    target = df["death_event"]


df = pd.concat([features, target], axis=1)
df

,age,serum_creatinine,ejection_fraction,death_event
0,75.0,1.9,20,1
1,55.0,1.1,38,1
2,65.0,1.3,20,1
3,50.0,1.9,20,1
4,65.0,2.7,20,1
...,...,...,...,...
294,62.0,1.1,38,0
295,55.0,1.2,38,0
296,45.0,0.8,60,0
297,45.0,1.4,38,0


In [213]:
counts = target.value_counts()
total = len(target)

for label, count in counts.items():
    percent = (count / total) * 100
    print(f"Liczba {'zmarłych' if label == (1,) else 'żywych'}: {count} ({percent:.2f}%)")

Liczba żywych: 203 (67.89%)
Liczba zmarłych: 96 (32.11%)


## Zastosowanie modelu

In [4]:
X = df[["age", "serum_creatinine", "ejection_fraction"]]
y = df["death_event"]

In [215]:
results = []

for i in range(100):
    # Podział danych na zbiór treningowy i testowy z random_state = iteracja, aby za każdym razem podział był inny
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i
    )

    # Standaryzacja danych (treningowych i testowych)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(
        X_test
    )

    # Tworzenie modelu regresji logistycznej
    model = LogisticRegression()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Obliczanie metryk
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append(
        {
            "iteration": i,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
        }
    )

results_df = pd.DataFrame(results)

results_df

,iteration,accuracy,precision,recall,f1_score
0,0,0.722222,0.588235,0.357143,0.444444
1,1,0.777778,0.687500,0.423077,0.523810
2,2,0.822222,0.722222,0.541667,0.619048
3,3,0.755556,0.640000,0.551724,0.592593
4,4,0.788889,0.692308,0.375000,0.486486
...,...,...,...,...,...
95,95,0.788889,0.750000,0.517241,0.612245
96,96,0.777778,0.666667,0.461538,0.545455
97,97,0.722222,0.642857,0.545455,0.590164
98,98,0.777778,0.666667,0.461538,0.545455


#### Ewaluacja modeli

In [216]:
stats_results = results_df.aggregate(["min", "max", "mean"]).round(2)
stats_results = stats_results.drop(columns=["iteration"])
stats_results.transpose()

,min,max,mean
accuracy,0.66,0.84,0.76
precision,0.45,0.93,0.68
recall,0.25,0.67,0.46
f1_score,0.35,0.70,0.54


#### Optymalizacja modelu

- Wyszukiwanie najlepszych parametrów z GridSearch

Metoda GridSearchCV jest jedną z popularnych technik optymalizacji modeli regresji logistycznej, automatyzującą wyszukiwanie najlepszych hiperparametrów, takich jak siła i typ regularyzacji. Poprawia wydajność modelu poprzez użycie walidacji krzyżowej, zapewniając solidność i możliwość uogólnienia na nowe dane.

`Testowane parametry:`
- penalty: określa rodzaj regularyzacji, która ma być zastosowana do modelu. Regularyzacja pomaga zapobiegać przeuczeniu (overfitting), dodając karę za duże wartości współczynników
- C: parametr odwrotności siły regularyzacji. Jest to liczba zmieniająca wpływ kary za duże wartości współczynników
- class_weight: parametr ten pozwala przypisać różne wagi klasom w celu zrównoważenia ich wpływu na model. Jest szczególnie przydatny, gdy mamy nierównomierny rozkład klas (np. jedna klasa występuje dużo częściej niż druga)

In [217]:
results = []

for i in range(100):
    # Podział danych na zbiór treningowy i testowy z random_state = iteracja, aby za każdym razem podział był inny
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i
    )

    # Standaryzacja danych (treningowych i testowych)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(
        X_test
    ) 

    # Tworzenie modelu regresji logistycznej
    model = LogisticRegression(solver="liblinear")

    param_grid = [
    {
        "penalty": [
            "l1",
            "l2",
        ],
        "C": [0.01, 0.1, 1.0, 10.0],
        "class_weight": [None,
            "balanced",
            {0: 1, 1: 2},
        ],
    }
]

    grid_search = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring="recall")

    grid_search.fit(X_train, y_train)

    # Obliczanie metryk
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append({"best_params": grid_search.best_params_})

results_gs = pd.DataFrame(results)
results_gs

,best_params
0,"{'C': 0.1, 'class_weight': 'balanced', 'penalt..."
1,"{'C': 0.1, 'class_weight': 'balanced', 'penalt..."
2,"{'C': 0.1, 'class_weight': 'balanced', 'penalt..."
3,"{'C': 0.1, 'class_weight': {0: 1, 1: 2}, 'pena..."
4,"{'C': 0.01, 'class_weight': {0: 1, 1: 2}, 'pen..."
...,...
95,"{'C': 0.1, 'class_weight': 'balanced', 'penalt..."
96,"{'C': 0.1, 'class_weight': {0: 1, 1: 2}, 'pena..."
97,"{'C': 0.1, 'class_weight': 'balanced', 'penalt..."
98,"{'C': 0.01, 'class_weight': 'balanced', 'penal..."


In [218]:
best_params_list = [json.dumps(result["best_params"]) for result in results]

counter = Counter(best_params_list)

# Sortowanie wyników od najczęstszych do najmniej częstych
sorted_results = sorted(counter.items(), key=lambda x: x[1], reverse=True)

print("Najczęściej występujące najlepsze parametry:")
for params, count in sorted_results[:3]:
    print(f"Parametry: {params} - Wystąpienia: {count}")

Najczęściej występujące najlepsze parametry:
Parametry: {"C": 0.1, "class_weight": "balanced", "penalty": "l1"} - Wystąpienia: 43
Parametry: {"C": 0.01, "class_weight": "balanced", "penalty": "l2"} - Wystąpienia: 33
Parametry: {"C": 0.1, "class_weight": {"0": 1, "1": 2}, "penalty": "l1"} - Wystąpienia: 10


In [5]:
results = []

for i in range(100):
    # Podział danych na zbiór treningowy i testowy z random_state = iteracja, aby za każdym razem podział był inny
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i
    )

    # Standaryzacja danych (treningowych i testowych)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(
        X_test
    )

    # Tworzenie modelu regresji logistycznej
    model = LogisticRegression(solver="liblinear", C=0.1, class_weight="balanced", penalty="l1")

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Obliczanie metryk
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append(
        {
            "iteration": i,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
        }
    )

results_df = pd.DataFrame(results)

stats_results = results_df.aggregate(["min", "max", "mean"]).round(2)
stats_results = stats_results.drop(columns=["iteration"])
stats_results.transpose()

,min,max,mean
accuracy,0.57,0.82,0.69
precision,0.35,0.69,0.52
recall,0.54,0.90,0.73
f1_score,0.49,0.74,0.60
